# Training Example

## Imports

In [27]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [28]:
import torch.nn as nn
from torch.optim import Adam
from project.scripts.models import get_model
from torch.optim.lr_scheduler import CosineAnnealingLR
import timm

## Training

In [29]:
num_classes = 16
num_epochs = 1


In [31]:
model = get_model('inception_v3.tv_in1k')

In [ ]:
# get model specific transforms (normalization, resize)
data_config = timm.data.resolve_model_data_config(model)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer_ = Adam(model.parameters(), lr=0.0005)
scheduler_ = CosineAnnealingLR(
    optimizer_,
    T_max=(len(train_loader.dataset) * num_epochs) // train_loader.batch_size,
)